<a href="https://colab.research.google.com/github/vchellapilla/Capstone3/blob/main/C3_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone 3: Modeling

## Import

Import packages.

In [1]:
! pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import scipy.stats as stats
import sklearn as sk
import surprise

Import datasets.

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls -l /content/drive/MyDrive/Colab\ Notebooks/Capstone\ 3/df_ppd_overall.csv

-rw------- 1 root root 6238405736 Sep 26 14:12 '/content/drive/MyDrive/Colab Notebooks/Capstone 3/df_ppd_overall.csv'


In [5]:
!ls -l /content/drive/MyDrive/Colab\ Notebooks/Capstone\ 3/df_ppd2.csv

-rw------- 1 root root 849468497 Sep 26 14:12 '/content/drive/MyDrive/Colab Notebooks/Capstone 3/df_ppd2.csv'


In [6]:
path_overall = '/content/drive/MyDrive/Colab Notebooks/Capstone 3/df_ppd2.csv'
df = pd.read_csv(path_overall, lineterminator='\n')

## Initial Looks

Let's take an initial look at our datasets.

In [7]:
df.head()

,Unnamed: 0,movie_id,user_id,user_rating_for_movie,u_num_ratings_pages,u_num_reviews,movie_popularity,days_since_movie_release,movie_runtime,avg_votes_for_movie,vote_count_for_movie,has_image
0,0,feast-2014,deathproof,7,32.0,1650.0,9.260,2879.0,6.0,7.9,720.0,1
1,1,loving-2016,deathproof,7,32.0,1650.0,18.024,2138.0,123.0,6.7,759.0,1
2,2,scripted-content,deathproof,7,32.0,1650.0,1.400,3025.0,2.0,6.8,5.0,1
3,3,the-future,deathproof,4,32.0,1650.0,5.208,4063.0,91.0,6.0,60.0,1
4,4,mank,deathproof,5,32.0,1650.0,16.331,668.0,132.0,6.9,1077.0,1


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10415253 entries, 0 to 10415252
Data columns (total 12 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Unnamed: 0                int64  
 1   movie_id                  object 
 2   user_id                   object 
 3   user_rating_for_movie     int64  
 4   u_num_ratings_pages       float64
 5   u_num_reviews             float64
 6   movie_popularity          float64
 7   days_since_movie_release  float64
 8   movie_runtime             float64
 9   avg_votes_for_movie       float64
 10  vote_count_for_movie      float64
 11  has_image                 int64  
dtypes: float64(7), int64(3), object(2)
memory usage: 953.5+ MB


In [9]:
df = df.drop('Unnamed: 0', axis=1)

In [10]:
df.isna().sum()

movie_id                      0
user_id                       0
user_rating_for_movie         0
u_num_ratings_pages           0
u_num_reviews                 0
movie_popularity              0
days_since_movie_release    196
movie_runtime                 0
avg_votes_for_movie           0
vote_count_for_movie          0
has_image                     0
dtype: int64

In [11]:
df.dropna(subset=['days_since_movie_release'], inplace=True)

## Train-Test Split

((why train test split. what is it))
<br><br>
((we will be splitting based on users so that the test set is a dataset of users unseen by the model.))

In [12]:
np.random.seed(21)
df1 = df.sample(n=50000)

In [13]:
from surprise import Reader, Dataset

reader = Reader(line_format = 'user item rating', rating_scale=(1, 10))
data = Dataset.load_from_df(df1[['user_id', 'movie_id', 'user_rating_for_movie']], reader)

In [14]:
from surprise.model_selection import train_test_split

#split
train_set, test_set = train_test_split(data, test_size =0.3, random_state=23)

In [15]:
#https://surprise.readthedocs.io/en/stable/trainset.html
#traindf = train_set.build_full_trainset()
#testdf = test_set.build_full_trainset()

## Model

((what is a user-based collaboriatve model. memory-based. what are the steps involved))

In [16]:
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold

((these are models we'll be trying))

explain model

In [17]:
#import KNNBasic
from surprise import KNNBasic

#define a cross-validation iterator
kfolds = KFold(n_splits=5)

#instantiate
sim_options = {'name' : 'cosine', 'user_based': True}
knnbasic = KNNBasic(k=50, sim_options = sim_options, random_state=23)

for trainset, testset in kfolds.split(data):

    #train and test algorithm
    knnbasic.fit(train_set)
    knnbasic_pred = knnbasic.test(test_set)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(knnbasic_pred, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.0906
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.0906
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.0906
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.0906
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.0906


explain model

In [18]:
#import KNNWithMeans
from surprise import KNNWithMeans

#define a cross-validation iterator
kfolds = KFold(n_splits=5)

#instantiate
sim_options = {'name' : 'cosine', 'user_based': True}
knnwm = KNNWithMeans(k=50, sim_options = sim_options, random_state=23)

for trainset, testset in kfolds.split(data):

    #train and test algorithm
    knnwm.fit(train_set)
    knnwm_pred = knnwm.test(test_set)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(knnwm_pred, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.1552
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.1552
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.1552
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.1552
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.1552


explain model

In [19]:
#import SVD
from surprise import SVD

#define a cross-validation iterator
kfolds = KFold(n_splits=5)

#instantiate
svd = SVD(random_state=23)

for trainset, testset in kfolds.split(data):

    #train and test algorithm
    svd.fit(train_set)
    svd_pred = svd.test(test_set)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(svd_pred, verbose=True)

RMSE: 1.8764
RMSE: 1.8764
RMSE: 1.8764
RMSE: 1.8764
RMSE: 1.8764


explain model

In [20]:
#import SVD++
from surprise import SVDpp

#define a cross-validation iterator
kfolds = KFold(n_splits=5)

#instantiate
svdpp = SVDpp(random_state=23)

for trainset, testset in kfolds.split(data):

    #train and test algorithm
    svdpp.fit(train_set)
    svdpp_pred = svdpp.test(test_set)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(svdpp_pred, verbose=True)

RMSE: 1.8598
RMSE: 1.8598
RMSE: 1.8598
RMSE: 1.8598
RMSE: 1.8598


explain model

In [21]:
#import NMF
from surprise import NMF

#define a cross-validation iterator
kfolds = KFold(n_splits=5)

#instantiate
nmf = NMF(random_state=23)

for trainset, testset in kfolds.split(data):

    #train and test algorithm
    nmf.fit(train_set)
    nmf_pred = nmf.test(test_set)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(nmf_pred, verbose=True)

RMSE: 2.3849
RMSE: 2.3849
RMSE: 2.3849
RMSE: 2.3849
RMSE: 2.3849


explain model

explain model

In [22]:
#import CoClustering
from surprise import CoClustering

#define a cross-validation iterator
kfolds = KFold(n_splits=5)

#instantiate
co = CoClustering(random_state=23)

for trainset, testset in kfolds.split(data):

    #train and test algorithm
    co.fit(train_set)
    co_pred = co.test(test_set)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(co_pred, verbose=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  del sys.path[0]


RMSE: 2.0509
RMSE: 2.0509
RMSE: 2.0509
RMSE: 2.0509
RMSE: 2.0509


Thus, SVD++ had the lowest RMSE scores across the board.

Let's improve on SVD++. ((hyperparameter tuning))

In [23]:
from surprise.model_selection import GridSearchCV

grid = {'n_factors': [20, 50],
        'n_epochs': [20, 50],
        'lr_all': [0.002, 0.005],
        'reg_all': [0.4, 0.6]}

gs_svdpp = GridSearchCV(algo_class=SVDpp, param_grid=grid, n_jobs=-1, cv=5, refit=True)
gs_svdpp.fit(data) #https://github.com/NicolasHug/Surprise/issues/190

In [24]:
gs_svdpp.best_params

{'rmse': {'n_factors': 20, 'n_epochs': 50, 'lr_all': 0.005, 'reg_all': 0.4},
 'mae': {'n_factors': 20, 'n_epochs': 50, 'lr_all': 0.005, 'reg_all': 0.4}}

In [25]:
gs_svdpp.best_estimator

{'rmse': <surprise.prediction_algorithms.matrix_factorization.SVDpp at 0x7f35966ea510>,
 'mae': <surprise.prediction_algorithms.matrix_factorization.SVDpp at 0x7f35966eaad0>}

In [30]:
#use the model that yields the best rmse
algo = gs_svdpp.best_estimator['rmse']
algo.fit(train_set)

In [32]:
#predict on that model for the test_set
pred_algo = algo.test(test_set)

In [36]:
pred_algo

[Prediction(uid='nosnevets', iid='the-wedding-ringer', r_ui=7.0, est=6.117240803600048, details={'was_impossible': False}),
 Prediction(uid='12pt9', iid='felix-the-ghost-breaker', r_ui=6.0, est=7.411574973742871, details={'was_impossible': False}),
 Prediction(uid='davidehrlich', iid='elle', r_ui=8.0, est=6.070945996977369, details={'was_impossible': False}),
 Prediction(uid='peng', iid='for-sama', r_ui=8.0, est=7.378640347890618, details={'was_impossible': False}),
 Prediction(uid='tylergfoster', iid='the-pink-panther', r_ui=5.0, est=5.273892415797855, details={'was_impossible': False}),
 Prediction(uid='kikoto', iid='lemony-snickets-a-series-of-unfortunate-events', r_ui=7.0, est=6.1908956607463015, details={'was_impossible': False}),
 Prediction(uid='oscarrosales', iid='shakespeare-in-love', r_ui=9.0, est=7.412682976578773, details={'was_impossible': False}),
 Prediction(uid='weardjupiter', iid='vincent-gallo-as-flying-christ', r_ui=6.0, est=7.315532661750793, details={'was_impossibl

In [66]:
#custom function to get recommendations

#function:
#input: take in a user_id and a movie_id
#output: spit out top 5 movie recs (5 nearest 5 movie ids)

#https://surprise.readthedocs.io/en/stable/FAQ.html

from collections import defaultdict

def get_movie_recs(user, predictions):
  """ Return top 5 movie recommendations for each user from sampled dataset.

      Args:
        predicitons: The list of predictions returned from the algorithm tested.
        user: 

      Returns:

  """
for user in predictions:
  try:
    top = defaultdict(list)

    #first, map the predicitons to each user
    for uid, iid, true_r, est, _ in predictions:
      top[uid].append((iid, est))
    
    #then, sort the predicitons for each user and get the top 5
    for uid, user_ratings in top.items():
      user_ratings.sort(key=lambda x: x[1], reverse=True)
      top[uid] = user_ratings[:4]
    
  return top

  except ValueError: #if an invalid entry was fed to the function
    print("Please enter a valid User ID and Movie ID.")

recs = get_movie_recs(pred_algo)

for uid, user_ratings in recs.items():
    print(uid, [iid for (iid, _) in user_ratings])

SyntaxError: ignored

In [63]:
get_movie_recs('dominichemphill', gs_svdpp)

TypeError: ignored

In [70]:
#custom function to get recommendations

#function:
#input: take in a user_id and a movie_id
#output: spit out top 5 movie recs (5 nearest 5 movie ids)

#https://surprise.readthedocs.io/en/stable/FAQ.html

from collections import defaultdict

def get_movie_recs(predictions):
  """ Return top 5 movie recommendations for each user from sampled dataset.

      Args:
        predicitons: The list of predictions returned from the algorithm tested.
        user: 

      Returns:

  """
  try:
    top = defaultdict(list)

    #first, map the predicitons to each user
    for uid, iid, true_r, est, _ in predictions:
      top[uid].append((iid, est))
    
    #then, sort the predicitons for each user and get the top 5
    for uid, user_ratings in top.items():
      user_ratings.sort(key=lambda x: x[1], reverse=True)
      top[uid] = user_ratings[:4]
    
    return top

  except ValueError: #if an invalid entry was fed to the function
    print("Please enter a valid User ID and Movie ID.")

recs = get_movie_recs(pred_algo)

for uid, user_ratings in recs.items():
    print(uid, [iid for (iid, _) in user_ratings])

nosnevets ['the-wedding-ringer']
12pt9 ['five-element-ninjas', 'felix-the-ghost-breaker', 'giallo-in-venice', 'promenade-du-dragon-a-cholon-i']
davidehrlich ['capernaum', 'elle', 'holmes-watson']
peng ['for-sama', 'ad-astra-2019', 'bait-2019']
tylergfoster ['hunt-for-the-wilderpeople', 'bring-me-the-head-of-alfredo-garcia', 'support-the-girls', 'my-cousin-vinny']
kikoto ['marvel-one-shot-all-hail-the-king', 'lemony-snickets-a-series-of-unfortunate-events']
oscarrosales ['the-warriors', 'shakespeare-in-love', 'gnomeo-juliet']
weardjupiter ['les-cousins', 'vincent-gallo-as-flying-christ', 'cosmic-ray', 'all-souls-day-dia-de-los-muertos']
dirtoddhaynes ['queen-of-katwe', 'lovesong-2016']
inoelshikari ['language-lessons', 'long-shot-2019', 'kingsman-the-golden-circle']
jeremyratzlaff ['the-triplets-of-belleville', 'krampus', 'lucy-2014']
scottstreet ['lethal-weapon', 'me-myself-irene', 'what-a-girl-wants', 'epic']
gaymafia ['kiss-the-girls', 'black-christmas-2006']
mayaderen_fan ['end-1992